# Part I. ETL Pipeline for Pre-Processing the Files

#### Importing Python packages

In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking current working directory
print(os.getcwd())

# Getting current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Creating a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# joining the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
print(file_path_list)

/home/workspace
['/home/workspace/event_data/2018-11-30-events.csv', '/home/workspace/event_data/2018-11-23-events.csv', '/home/workspace/event_data/2018-11-22-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-11-events.csv', '/home/workspace/event_data/2018-11-14-events.csv', '/home/workspace/event_data/2018-11-20-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-05-events.csv', '/home/workspace/event_data/2018-11-28-events.csv', '/home/workspace/event_data/2018-11-25-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-24-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-12-events.csv', '/home/workspace/event_data/2018-11-27-events.c

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
        # extracting each data row one by one and append to full_data_rows_list     
        for line in csvreader:
            #print(line)
            #Each line is a list of strings and full_data_rows_list is a list of lists
            full_data_rows_list.append(line) 
            
print(len(full_data_rows_list))
#print(full_data_rows_list)

# creating one event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables. This csv file has data from all the source csv files
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Creation and populating of Cassandra tables based on query requirements


#### Cluster creation

In [5]:
# making a connection to a Cassandra instance in local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, creating a session object from cluster object
session = cluster.connect()

#### Keyspace creation

In [6]:
# Creating a Keyspace using the method session.execute
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS kkd 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Setting KEYSPACE to 'kkd' for the session
try:
    session.set_keyspace('kkd')
except Exception as e:
    print(e)

### Query 1: Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [8]:
# Creating 'song_session_iteminsession' table for Query 1
# sessionID should be the partition key and iteminSession should be the clustering column
# (sessionID, iteminSession) is unique primary key for this table
# The table columns are based on primary key and Query 1 requirement

query = "DROP TABLE IF EXISTS song_session_iteminsession"
try:
    session.execute(query)
except Exception as e:
    print(e)

query = "CREATE TABLE IF NOT EXISTS song_session_iteminsession"
query = query + "(sessionid int, iteminsession int, artist text, song text, length float, PRIMARY KEY (sessionid, iteminsession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

       
# Reading from event_datafile_new.csv and writing into the table song_session_iteminsession
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:  # Each line is a list of strings
        #print(line) 
        query = "INSERT INTO song_session_iteminsession (sessionid, iteminsession, artist, song, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        # In the csv file, artist in col 0, firstname in col 1, gender in col 2, iteminsession in col 3, 
        # lastname in col 4, length in col 5, level in col 6, location in col 7, 
        # sessionid in col 8, song in col 9, userid in col 10
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))
        
# Verifying the data was correctly entered into the table song_session_iteminsession
# The output should give artist, song name / length from a specific sessionid / iteminSession
query = "SELECT artist, song, length FROM song_session_iteminsession WHERE sessionid=338 AND iteminsession=4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
for row in rows:
    print(row.artist, ",", row.song, ",", row.length)

Faithless , Music Matters (Mark Knight Dub) , 495.30731201171875


### Query 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [9]:
# Creating 'song_playlist_session' table for Query 2
# (userID, sessionID) is the composite partition key and iteminSession is clustering column as per query / sort requirement
# ((userID, sessionID), iteminSession) is the unique primary key 
# The table columns are based on primary key and Query 2 requirement

query = "DROP TABLE IF EXISTS song_playlist_session"
try:
    session.execute(query)
except Exception as e:
    print(e)

query = "CREATE TABLE IF NOT EXISTS song_playlist_session"
query = query + "(userid int, sessionid int, iteminsession int, artist text, song text, user_first text, user_last text, \
                    PRIMARY KEY ((userid, sessionid), iteminsession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

    
# Reading from event_datafile_new.csv and writing into the table song_playlist_session
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:  ## Each line is a list of strings
        query = "INSERT INTO song_playlist_session (userid, sessionid, iteminsession, artist, song, user_first, user_last)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        # In the csv file, artist in col 0, firstname in col 1, gender in col 2, iteminsession in col 3, 
        # lastname in col 4, length in col 5, level in col 6, location in col 7, 
        # sessionid in col 8, song in col 9, userid in col 10
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))
                    
            
# Verifying the data was correctly entered into the table song_playlist_session
# The output should give artists, songs and user name from a specific userid / sessionid

query = "SELECT artist, song, user_first, user_last FROM song_playlist_session WHERE userid=10 AND sessionid=182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
for row in rows:
    print(row.artist, ",", row.song, ",", row.user_first, row.user_last)

Down To The Bone , Keep On Keepin' On , Sylvie Cruz
Three Drives , Greece 2000 , Sylvie Cruz
Sebastien Tellier , Kilometer , Sylvie Cruz
Lonnie Gordon , Catch You Baby (Steve Pitron & Max Sanna Radio Edit) , Sylvie Cruz


### Query 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [10]:
# Creating 'user_list_song' table for Query 3
# song is the partition key and userid is clustering column as per query requirement
# It is to be noted that users may have the same name (first or last) and so userid is used as the clustering column
# (song, userid) is the unique primary key 
# The table columns are based on primary key and Query 3 requirement

query = "DROP TABLE IF EXISTS user_list_song"
try:
    session.execute(query)
except Exception as e:
    print(e)

query = "CREATE TABLE IF NOT EXISTS user_list_song"
query = query + "(song text, userid int, user_first text, user_last text, \
                    PRIMARY KEY (song, userid))"

try:
    session.execute(query)
except Exception as e:
    print(e)

# Reading from event_datafile_new.csv and writing into the table user_list_song
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader: # Each line is a list of strings
        query = "INSERT INTO user_list_song (song, userid, user_first, user_last)"
        query = query + "VALUES (%s, %s, %s, %s)"
        # In the csv file, artist in col 0, firstname in col 1, gender in col 2, iteminsession in col 3, 
        # lastname in col 4, length in col 5, level in col 6, location in col 7, 
        # sessionid in col 8, song in col 9, userid in col 10
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))
                    
           
# Verifying the data was correctly entered into the table song_playlist_session
# The output should give user names of all users who listened to the song 'All Hands Against His Own'
query = "SELECT user_first, user_last FROM user_list_song WHERE song='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
for row in rows:
    print(row.user_first, row.user_last)

                    

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Dropping the tables before closing out the sessions

In [11]:
query = "DROP TABLE IF EXISTS song_session_iteminsession"
try:
    session.execute(query)
except Exception as e:
    print(e)
    
query = "DROP TABLE IF EXISTS song_playlist_session"
try:
    session.execute(query)
except Exception as e:
    print(e)
    
query = "DROP TABLE IF EXISTS user_list_song"
try:
    session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [12]:
session.shutdown()
cluster.shutdown()